In [27]:
#!/usr/bin/env python3
import os
import json
from datetime import datetime

# --- CONFIG ---
SNAP_DIR = "snapshots"
TIME_FMT = "%Y%m%dT%H%M%SZ"  # matches your snapshot timestamps
# ---------------

def load_snapshots(dirpath):
    """
    Return a list of (timestamp: datetime, units: dict of apt → data),
    sorted by timestamp ascending.
    """
    files = sorted(
        fn for fn in os.listdir(dirpath)
        if fn.startswith("snapshot_") and fn.endswith(".json")
    )
    snapshots = []
    for fn in files:
        path = os.path.join(dirpath, fn)
        with open(path) as f:
            doc = json.load(f)
        # Use the embedded timestamp (in case you renamed files differently)
        ts = datetime.strptime(doc["timestamp"], TIME_FMT)
        snapshots.append((ts, doc["units"]))
    return snapshots

def build_presence_map(snapshots):
    """
    Map each apartment_number → list of timestamps in which it appeared.
    """
    presence = {}
    for ts, units in snapshots:
        for apt_num in units.keys():
            presence.setdefault(apt_num, []).append(ts)
    return presence

def print_timeline(snapshots):
    print("📊 Units per snapshot:")
    for ts, units in snapshots:
        # sort numerically when possible
        apt_list = sorted(
            units.keys(),
            key=lambda x: int(x) if x.isdigit() else x
        )
        count = len(apt_list)
        print(f"  {ts.date()}: {count} units — {', '.join(apt_list)}")

def print_active_vs_rented(presence, snapshots):
    first_ts = snapshots[0][0]
    last_ts  = snapshots[-1][0]

    still_active = [apt for apt, dates in presence.items() 
                    if last_ts in dates]
    dropped      = [apt for apt, dates in presence.items() 
                    if last_ts not in dates]

    print(f"\n✅ Still active ({len(still_active)}):")
    print("  " + ", ".join(sorted(still_active)))

    print(f"\n❌ Rented / removed since first snapshot ({len(dropped)}):")
    print("  " + ", ".join(sorted(dropped)))

def print_uptime_for_active(presence, snapshots):
    """
    For each apt still active, compute days between first-seen and last-snapshot.
    """
    last_snap = snapshots[-1][0]
    print("\n⏱ Uptime for still-active apartments:")
    for apt in sorted(presence.keys()):
        dates = presence[apt]
        if last_snap in dates:
            first_seen = dates[0]
            days_up = (last_snap - first_seen).days
            print(f"  Apt {apt}: first seen {first_seen.date()} → up for {days_up} day(s)")

def print_sold_summary(snapshots):
    """
    Print a summary of any apartments that were in the first snapshot
    but no longer in the last one.
    """
    if not snapshots:
        print("No snapshots to analyze.")
        return

    # assume snapshots is a list of (timestamp, units_dict) sorted by timestamp
    first_ts, first_units = snapshots[0]
    last_ts,  last_units  = snapshots[-1]

    sold = sorted(
        set(first_units) - set(last_units),
        key=lambda x: int(x) if x.isdigit() else x
    )

    if not sold:
        print("✅ No apartments have been sold (i.e. none disappeared).")
        return

    print("\n🏷️  Apartments Sold Summary:")
    header = f"{'Apt':<6}{'Last Rent':<16}{'Move‑In':<12}{'Last Seen'}"
    print(header)
    print("-" * len(header))

    for apt in sold:
        # find the last snapshot in which this apt was present
        for ts, units in reversed(snapshots):
            if apt in units:
                u = units[apt]
                rent    = u.get("price_display", "—")
                move_in = u.get("available_display", "—")
                last_seen = ts.date()
                print(f"{apt:<6}{rent:<16} {move_in:<12} {last_seen}")
                break

def print_available_summary(snapshots):
    """
    Print a summary of any apartments that are present in the last snapshot.
    Shows Apt, current rent, move‑in date, and the first date they appeared.
    """
    if not snapshots:
        print("No snapshots to analyze.")
        return

    # assume snapshots is a list of (timestamp, units_dict) sorted by timestamp
    last_ts, last_units = snapshots[-1]

    available = sorted(last_units.keys(), key=lambda x: int(x) if x.isdigit() else x)
    if not available:
        print("ℹ️  No apartments available in the last snapshot.")
        return

    print("\n✅ Apartments Still Available:")
    header = f"{'Apt':<6}{'Rent':<16}{'Move‑In':<12}{'First Seen'}"
    print(header)
    print("-" * len(header))

    for apt in available:
        u = last_units[apt]
        rent     = u.get("price_display", "—")
        move_in  = u.get("available_display", "—")

        # find the first snapshot where this apt appears
        first_seen = None
        for ts, units in snapshots:
            if apt in units:
                first_seen = ts.date()
                break

        print(f"{apt:<6}{rent:<16} {move_in:<12} {first_seen}")

In [28]:
snaps = load_snapshots(SNAP_DIR)
if not snaps:
    print("No snapshots found in", SNAP_DIR)
    
presence = build_presence_map(snaps)
print_timeline(snaps)
print_active_vs_rented(presence, snaps)
print_uptime_for_active(presence, snaps)
print_sold_summary(snaps)
print_available_summary(snaps)

📊 Units per snapshot:
  2025-07-14: 5 units — 221, 265, 409, 511, 670
  2025-07-15: 4 units — 265, 409, 511, 670
  2025-07-16: 4 units — 265, 409, 511, 670
  2025-07-17: 4 units — 265, 409, 511, 670
  2025-07-18: 3 units — 265, 409, 511
  2025-07-19: 3 units — 265, 409, 511
  2025-07-20: 3 units — 265, 409, 511
  2025-07-21: 3 units — 265, 409, 511
  2025-07-22: 3 units — 265, 409, 511

✅ Still active (3):
  265, 409, 511

❌ Rented / removed since first snapshot (2):
  221, 670

⏱ Uptime for still-active apartments:
  Apt 265: first seen 2025-07-14 → up for 7 day(s)
  Apt 409: first seen 2025-07-14 → up for 7 day(s)
  Apt 511: first seen 2025-07-14 → up for 7 day(s)

🏷️  Apartments Sold Summary:
Apt   Last Rent       Move‑In     Last Seen
-------------------------------------------
221   Base Rent $2,250 Available Aug 08 2025-07-14
670   Base Rent $2,290 Available Jul 30 2025-07-17

✅ Apartments Still Available:
Apt   Rent            Move‑In     First Seen
-----------------------------